# Almost Perfect: A Discussion on Quasi-Experiments Techniques

Quasi-experiments are experiments that leverage the principal from randomized tests, but are not equivalent

Any technique that can be used to estimate causal effects from observational data can be used to extract the causal effect from an quasi-experiment. The use of these causal inferences techniques in quasi-experiments is the reduction in the variance and bias of the calculated ATT (or ATE), similarly to the effect these techniques in randomized experiments. 

However, one of the biggest problems with using causal inference techniques is that they inevevitably rely on assumptions about the causal links between variables. While there are advancements in causal discovery, in practice one never consider all possible configurations between cofounding, treatment, and target variables. Instead, we basically always create a Directed Acyclic Graph (DAG) to lay out the causal relationships in such way that the scientists behind, their piers, and clients are satisfied with.



"CUPED is just linear regression using a pre-experimental covariate."[2]

Following, we give a quick overview of the methods we cover in this benchmark, for a better in-depth understading of each method, we provide multiple contents where you can learn more about them

## Matching + Differences-in-Differences (CausalPy)

### Propensity Score 

### Mahalanobis Distance

## (Augmented) Synthetic Control (CausalPy & GeoLift)

## Meta-Learners (CausalML)
    
## Double ML (EconML)

## Uplift-Trees (CausalML)

## Do Method (DoWhy)



# References
1) [How Booking.com increases the power of online experiments with CUPED](https://booking.ai/how-booking-com-increases-the-power-of-online-experiments-with-cuped-995d186fff1d)
2) [Variance reduction in experiments using covariate adjustment techniques](https://medium.com/glovo-engineering/variance-reduction-in-experiments-using-covariate-adjustment-techniques-717b1e450185)
3) [Causal Inference, The Mixtap](https://mixtape.scunning.com)
4) Causality, Judea Pearl
5) Causal Inference in Statistics, Judea Pearl, Madelyn Glymour, Nicholas P. Jewell
6) [CausalML](https://causalml.readthedocs.io/en/latest/index.html)
7) [EconML](https://econml.azurewebsites.net/index.html)
8) [CausalPy](https://causalpy.readthedocs.io/en/latest/)
9) [DoWhy](https://www.pywhy.org/dowhy/v0.11.1/#)

SyntaxError: invalid syntax (967379724.py, line 3)